[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/giswqs/notebook-share/blob/master/docs/dongting_lake_floods.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giswqs/notebook-share/HEAD?labpath=docs%dongting_lake_floods.ipynb)

# Visualization and Analysis of Alto-Garda Lake Flood 2020



## Introduction

On October 3, 2020, heavy rainfall led to severe flooding around Lake Garda, Italy's largest freshwater lake. The flooding resulted in the inundation of approximately 2.7 square miles of land. This event affected around buildings, agricultural fields,roads. Flood mapping can help in identifying the affected areas, planning emergency responses, and implementing effective recovery and mitigation strategies to reduce the impact of future flood events.




## Requirements

To follow this tutorial, you must first [sign up](https://earthengine.google.com/signup) for a [Google Earth Engine](https://earthengine.google.com/) account. Earth Engine is a cloud computing platform with a [multi-petabyte catalog](https://developers.google.com/earth-engine/datasets/) of satellite imagery and geospatial datasets. It is free for noncommercial use. To authenticate the Earth Engine Python API, see instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In this tutorial, we will use the [geemap](https://geemap.org) Python package to visualize and analyze the Garda Lake flood. Geemap enables users to analyze and visualize Earth Engine datasets interactively within a Jupyter-based environment with minimal coding.


## Installation

Uncomment the following line to install geemap if needed.

In [1]:
!pip install -U geemap

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.0 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Attempting uninstall: geemap
    Found existing installation: geemap 0.33.1
    Uninstalling geemap-0.33.1:
      Successfully uninstalled geemap-0.33.1


## Import libraries

Import the earthengine-api and geemap.

In [2]:
!pip install earthengine-api



In [3]:
import ee
import geemap.foliumap as geemap

In [4]:
import ee

# Authenticate the Google Earth Engine API
ee.Authenticate()

In [5]:
import ee

# Initialize the Google Earth Engine API with the specified project
ee.Initialize(project='eee-mashalahore') #specific name of project in GEE


## Create an interactive map

## Specify ROI and time range

In the tutorial, we will focus on the Garda Lake in Italy, but the code can be easily modified to visualize and analyze floods in other countries or regions. Modify the `geom` variable to specify the place of interest and set the date range for the flood event. In order to extract the flood extent, we also need to specify the date range for the pre-flood period.

In [59]:
m = geemap.Map(center=[41.8719, 12.5674], zoom=10) #define AOI with lat/long or insert shapefile
m

## Create Sentinel-1 SAR composites

We are use Sentinel-1 [Synthetic Aperture Radar (SAR)](https://www.earthdata.nasa.gov/learn/backgrounders/what-is-sar) data to extract flood extent. Radar can collect signals in different polarizations, by controlling the analyzed polarization in both the transmit and receive paths. Signals emitted in vertical (V) and received in horizontal (H) polarization would be indicated by a VH. Alternatively, a signal that was emitted in horizontal (H) and received in horizontal (H) would be indicated by HH, and so on. Examining the signal strength from these different polarizations carries information about the structure of the imaged surface. Rough surface scattering, such as that caused by bare soil or water, is most sensitive to VV scattering. Therefore, VV polarization is often used to detect water bodies.

Sentinel-1 operates in four exclusive [acquisition modes](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-1/instrument-payload):

* Stripmap (SM)
* Interferometric Wide swath (IW)
* Extra-Wide swath (EW)
* Wave mode (WV)

The Interferometric Wide swath (IW) mode allows combining a large swath width (250 km) with a moderate geometric resolution (5 m by 20 m). The IW mode is the default acquisition mode over land. In this tutorial, we will use Sentinel-1 IW mode data to extract flood extent.

The [Sentinel-1 SAR data](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S1_GRD) are available from 2014 to present. Let's filter the `COPERNICUS/S1_GRD` dataset by the date range and location.

In [7]:
pre_flood_start_date = "2020-09-01"
pre_flood_end_date = "2020-09-30"
post_flood_start_date = "2020-10-03"
post_flood_end_date = "2020-10-18"

In [35]:
# Create a map centered at a specific location.
m = geemap.Map()
m.add_basemap("HYBRID")

# Define the region of interest (ROI) with coordinates around the Garda area in Trento, northern Italy.
geom = ee.Geometry.Polygon(
    [
        [
            [10.396389, 45.990222],  # Expanded Point A (Northwest)
            [10.305, 45.458333],     # Expanded Point B (Southwest)
            [11.085, 45.485],        # Expanded Point C (Southeast)
            [11.163333, 45.995],     # Expanded Point D (Northeast)
            [10.396389, 45.990222],  # Back to Point A
        ]
    ]
)
roi = ee.FeatureCollection(geom)

# Load the JRC Global Surface Water dataset and clip to the ROI.
jrc = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").clipToCollection(roi)

# Visualization parameters for the water occurrence.
vis_params = {
    "bands": ["occurrence"],
    "min": 0.0,
    "max": 100.0,
    "palette": ["ffffff", "ffbbbb", "0000ff"],
}

# Add the layers to the map.
m.add_layer(jrc, vis_params, "Water Occurrence")
m.add_layer(geom, {}, "ROI")

# Center the map on the ROI.
m.center_object(geom)

# Display the map.
m


#Create the Sentinel-1 image collection for the pre-flood period.

In [36]:
s1_col_pre = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filterDate(pre_flood_start_date, pre_flood_end_date)
    .filterBounds(roi)
    .select("VV")
)
print(
    f"The number of images in the pre-flood collection: {s1_col_pre.size().getInfo()}"
)

The number of images in the pre-flood collection: 27


In [37]:
s1_col_post = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    # .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    .filterDate(post_flood_start_date, post_flood_end_date)
    .filterBounds(roi)
    .select("VV")
)
print(
    f"The number of images in the post-flood collection: {s1_col_post.size().getInfo()}"
)

The number of images in the post-flood collection: 14


In [38]:
m = geemap.Map()
m.add_basemap("HYBRID")
sar_pre = s1_col_pre.reduce(ee.Reducer.percentile([20])).clipToCollection(roi)
sar_post = s1_col_post.reduce(ee.Reducer.percentile([20])).clipToCollection(roi)
m.add_layer(sar_pre, {"min": -25, "max": -5}, "SAR Pre-flood")
m.add_layer(sar_post, {"min": -25, "max": -5}, "SAR Post-flood")
m.center_object(roi, 9)
m

In [39]:
s1_col_pre = s1_col_pre.map(lambda img: img.focal_median(100, "circle", "meters"))
s1_col_post = s1_col_post.map(lambda img: img.focal_median(100, "circle", "meters"))

In [40]:
m = geemap.Map()
m.add_basemap("HYBRID")
sar_pre = s1_col_pre.reduce(ee.Reducer.percentile([20])).clipToCollection(roi)
sar_post = s1_col_post.reduce(ee.Reducer.percentile([20])).clipToCollection(roi)
m.add_layer(sar_pre, {"min": -25, "max": -5}, "SAR Pre-flood")
m.add_layer(sar_post, {"min": -25, "max": -5}, "SAR Post-flood")
m.center_object(roi, 9)
m

In [41]:
m = geemap.Map()
left_layer = geemap.ee_tile_layer(sar_pre, {"min": -25, "max": -5}, "SAR Pre-flood")
right_layer = geemap.ee_tile_layer(sar_post, {"min": -25, "max": -5}, "SAR Post-flood")

m.split_map(
    left_layer,
    right_layer,
    left_label="Sentinel-1 Pre-flood",
    right_label="Sentinel-1 Post_flood",
)
m.center_object(roi, 9)
m

In [42]:
threshold = -16
water_pre = sar_pre.lt(threshold)
water_post = sar_post.lt(threshold)

In [43]:
m = geemap.Map()
m.add_layer(sar_pre, {"min": -25, "max": -5}, "SAR Pre-flood")
m.add_layer(sar_post, {"min": -25, "max": -5}, "SAR Post-flood")

left_layer = geemap.ee_tile_layer(
    water_pre.selfMask(), {"palette": "blue"}, "Water Pre-flood"
)
right_layer = geemap.ee_tile_layer(
    water_post.selfMask(), {"palette": "red"}, "Water Post-flood"
)

m.split_map(
    left_layer,
    right_layer,
    left_label="Water Pre-flood",
    right_label="Water Post-flood",
)
m.center_object(roi, 9)
m

In [44]:
flood_extent = water_post.subtract(water_pre).gt(0).selfMask()

In [45]:
m = geemap.Map()

m.add_layer(sar_pre, {"min": -25, "max": -5}, "SAR Pre-flood")
m.add_layer(sar_post, {"min": -25, "max": -5}, "SAR Post-flood")

left_layer = geemap.ee_tile_layer(
    water_pre.selfMask(), {"palette": "blue"}, "Water Pre-flood"
)
right_layer = geemap.ee_tile_layer(
    water_post.selfMask(), {"palette": "red"}, "Water Post-flood"
)

m.split_map(
    left_layer,
    right_layer,
    left_label="Water Pre-flood",
    right_label="Water Post-flood",
)

m.add_layer(flood_extent, {"palette": "cyan"}, "Flood Extent")
m.center_object(roi, 9)
m

In [46]:
area_pre_flood = geemap.zonal_stats(
    water_pre.selfMask(), roi, scale=30, stat_type="SUM", return_fc=True
)
area_pre_df = geemap.ee_to_df(area_pre_flood)
area_pre_df

Computing statistics ...


,sum
0,1.018867e+06


In [47]:
print(f"Pre-flood water area: {round(area_pre_df['sum'].values[0] * 900 / 1e6)} km2")

Pre-flood water area: 917 km2


In [48]:
area_post_flood = geemap.zonal_stats(
    water_post.selfMask(), roi, scale=30, stat_type="SUM", return_fc=True
)
area_post_df = geemap.ee_to_df(area_post_flood)
area_post_df

Computing statistics ...


,sum
0,990990.717647


In [49]:
print(f"Post-flood water area: {round(area_post_df['sum'].values[0] * 900 / 1e6)} km2")

Post-flood water area: 892 km2


In [50]:
flood_area = geemap.zonal_stats(
    flood_extent.selfMask(), roi, scale=30, stat_type="SUM", return_fc=True
)
flood_area_df = geemap.ee_to_df(flood_area)
flood_area_df

Computing statistics ...


,sum
0,51146.623529


In [51]:
print(f"Flooded area: {round(flood_area_df['sum'].values[0] * 900 / 1e6)} km2")

Flooded area: 46 km2
